In [1]:
# Advent of Code, Packet Decoder Day 16  
#
from rich.jupyter import print as rprint
from functools import partial
from math import prod
from pathlib import Path

DEBUG = False
"""
Reimplemented based on Rodrigo's 

Version # 0:2
Type    # 3:5
Length  # 6
    0: next 15 bits are a number that represents the total length of subpackets
    1: next 11 bits are a number that represents the total number fo subpackets
{11 or 15 bits in sub packets following}

Type 0: Sum
Type 1: Product
Type 2: Minimum
Type 3: Maximum
Type 4: Literal value, in four bit chunks, xyyy:
        x = continue numbers if 1, last number if 0
        yyy = three digit number
Type 5: >
Type 6: < 
Type 7: ==

    8A004A801A8002F478 operator packet (version 4) 
        which contains an operator packet (version 1) 
        which contains an operator packet (version 5) 
        which contains a literal value (version 6); 
        this packet has a version sum of 16.
    
    620080001611562C8802118E34 represents an operator packet (version 3) 
        which contains two sub-packets; 
        each sub-packet is an operator packet that contains two literal values. 
        This packet has a version sum of 12.
    
    C0015000016115A2E0802F182340 has the same structure as the previous example, 
        but the outermost packet uses a different length type ID. 
        This packet has a version sum of 23.
    
    A0016C880162017C3686B18A3D4780 is an operator packet 
        that contains an operator packet 
        that contains an operator packet 
        that contains five literal values; 
        it has a version sum of 31.
"""
def parse_packet(p):
    t, p = parse_type(p)
    version, p = parse_version(p)
    if version == "100":
        value, p = parse_value(p)
        return (int(t, 2), int(version, 2), int(value, 2)), p
    else:
        (flag, packets), p = parse_operator_packets(p)
        return (int(t, 2), int(version, 2), int(flag), packets), p

def parse_value(p):
    parsed = ""
    while p[0] == "1":
        group, p = parse_group(p[1:])
        parsed += group
    group, p = parse_group(p[1:])
    return parsed + group, p

def parse_operator_packets(p):
    flag = p[0]
    p = p[1:] 
    if flag == "0": # Bit
        length, p = parse_bit_length(p)
        bit_length = int(length, 2)
        to_parse = p[:bit_length]  # Part of the string to be parsed.
        packets = []
        while to_parse:
            packet, to_parse = parse_packet(to_parse)
            packets.append(packet)
        p = p[bit_length:]
    elif flag == "1":
        length, p = packet_length(p)
        packet_number = int(length, 2)
        packets = []
        for _ in range(packet_number):
            packet, p = parse_packet(p)
            packets.append(packet)
    return (flag, packets), p

# Parse n bytes, return remainder of packet
def parse_n(n, p):
    parsed = ""
    for _ in range(n):
        D = p[0]
        p = p[1:]
        parsed += D
    return parsed, p

parse_group = partial(parse_n, 4)
parse_bit_length = partial(parse_n, 15)
packet_length = partial(parse_n, 11)
parse_type = partial(parse_n, 3)
parse_version = partial(parse_n, 3)

In [2]:
INPUT_PATH = "input_files/day16.txt"
with open(INPUT_PATH, "r") as f:
    tmp = "".join(f.read().strip().splitlines())
p = "".join(f"{int(letter, 16):04b}" for letter in tmp)

transmission, leftovers = parse_packet(p)

def sum_versions(packet):
    sub = 0
    if len(packet) != 3:
        sub = sum(sum_versions(sub) for sub in packet[3])
    return packet[0] + sub

print(sum_versions(transmission))

866


In [3]:
eval_dict = {
    0: sum,
    1: prod,
    2: min,
    3: max,
    5: lambda packets: packets[0] > packets[1],
    6: lambda packets: packets[0] < packets[1],
    7: lambda packets: packets[0] == packets[1],
}

def evaluate(packet):
    if len(packet) == 3:
        return packet[2]
    subs = [evaluate(sub) for sub in packet[3]]
    return eval_dict[packet[1]](subs)

print(evaluate(transmission))

1392637195518
